In [1]:
import animal_breeding
import numpy as np
import pandas as pd
import plotly.express as px

*Note:* All code is to be found in the file **animal_breeding.py**.

# MDP with finite action and state space

## Toy problem: Running a farm

We consider the problem of running an animal farm where we either sell from the current stock or breed it. In relation to our problem, we consider the following qualities desirable
- All the animals may breed with each other,
- Per round the animals behave monogamously,
- For every pair of the current stock, with probability $p$ they can produce one offspring,
- At most $S_{\text{max}} \in \mathbb{N}$ animals may be present on the farm at any point in time,
- Reproducing above $S_{\text{max}}$ yields no increase in population.

In regards to choosing our reward function, we consider
- All animals go for the same price, but selling in bulk must be done at a discount.

For a bulk discount factor $d \in (0,1)$, we consider bulk discount function $s \mapsto (1-d)^{s - 1}$ where $s \le S_{\text{max}}$ is the stock sold. (Not to be confused with the discount factor of the infinite horizon problem.) We formulate the MDP problem as so:
- State space $E = \left\{ 0, \ldots, S_{\text{max}}\right\}$ representing the animal stock,
- Action space $A = \left\{ \text{W}, \text{B}, \text{Sell 1}, \ldots, \text{Sell }S_{\text{max}} \right\}$ where 
    - $A(0) = \left\{ \text{W} \right\}$,
    - $A(1) = \left\{ \text{W}, \text{Sell 1} \right\}$,
    - $A(x) = \left\{ \text{W}, \text{B}, \text{Sell 1}, \ldots, \text{Sell }x \right\}$ for $2 \le x \le S_{\text{max}}$.
- Action $\text{W}$ representing waiting, $\text{B}$ representing breeding $\lfloor x / 2 \rfloor$ animals, and $\text{Sell }x$ representing selling $x$ of the current stock,
- Reward function $r : X \times E \to \mathbb{R}$ where
    - $r(x, \text{W}) = -x \cdot \text{Marginal feeding cost}$ for any $x \in E$,
    - $r(x, \text{B}) = -x \cdot \text{Marginal feeding cost}$ for any $x \ge 2$,
    - $r(x, \text{Sell }y) = -(x-y) \cdot \text{Marginal feeding cost} +  (1-d)^{y-1}y \cdot \text{Fixed price}$ for $1 \le y \le x$.
- Transition probabilities $p ( \, \cdot \mid x, a) : E \to [0,1]$ where 
    - $p(y \mid x, \text{W}) = 1$ for $y = x$,
    - $p(y \mid x, \text{B}) = f_\text{Binomial}(y - x, \lfloor x / 2 \rfloor, p)$ for $x \le y < S_{\text{max}}$; $p(S_{\text{max}} \mid x, \text{B}) = P(\text{Binomial}(\lfloor x / 2 \rfloor, p) \ge S_{\text{max}})$,
        - *Quick comment*: We add the tail in the case where the current stock can reproduce an amount larger than the allowed maximal stock. That is, say we have a stock of 14 animals with a cap of 15 animals. Then choosing to breed the stock and receiving a stock of 15 animals is successly breeding $1, 2, \ldots, 6$ or $7$ animals.
    - $p(y \mid x, \text{Sell }s) = 1$ for $y-s=x$.

We implement a general class to handle the MDP problem.

In [2]:
class farm_problem():
    def __init__(
        self, 
        max_animals: int,
        animal_price: float, 
        feeding_cost: float,
        discount: float,
        p: float
    ):

        self.max_animals = max_animals
        self.animal_price = animal_price
        self.feeding_cost = feeding_cost 
        self.discount = discount
        self.discount_foo = lambda y: (1-self.discount)**(y-1)
        self.p = p

        self.states = range(0, max_animals+1)
        self.actions = dict()
        for state in self.states[1:]:
            if state > 1:
                self.actions[state] = ["W", "B", *range(1, state+1)]
            else:
                self.actions[state] = ["W", 1]
        self.actions[0] = ["W"]

        return

## Finite horizon problem

Consider the finite horizon problem for some horizon $T \in \mathbb{N}$ s.t. $T \ge \mathcal{E}$ and exit reward 
- $R(x) = (1-d)^{x-1}x$

for any $x \in E$. That is, at time $T$ we sell of the remaining stock if any. Recall the backward induction algorithm
- Set $t = N$ and let $U(T, x) = R(x)$ for $x \in E$,
- For $t = N - 1, \ldots, 1$ set 
$$
\textstyle
\pi_t(x) = \text{argmax}_{a \in A(x)} r(x,a) + \sum_{y \in E} p(y \mid x, a) U(t+1, y), \,
U(t, x) = r(x, \pi_t(x)) + \sum_{y \in E} p(y \mid x, \pi_t(x)) U(t+1, y),
$$
We know that $U = V^*$. The algorithm can be implemented as so

In [3]:
def bia(self, time_horizon: int = 20) -> list:
    T = time_horizon - 1

    u = np.zeros(shape = (time_horizon, len(self.states)), dtype = float)
    pi = np.empty(shape = (time_horizon - 1, len(self.states)), dtype = object)

    for t in [*range(time_horizon)][::-1]:
        print(t) 
        if t == T:
            for x in self.states:
                u[t, x] = self.R(x)
        else:
            for x in self.states:
                tmp = dict()
                for a in self.actions[x]:
                    tmp[a] = self.r(x, a)
                    for y in self.states:
                        tmp[a] += self.t(y, x, a) * u[t+1, y]  
                max_key = max(tmp, key=tmp.get)
                max_val = max(tmp.values())

                u[t, x] = max_val
                pi[t, x] = max_key
    
    return [pi, u]

An demonstration is in order. Consider

In [4]:
max_animals = 15 
animal_price = 10
feeding_cost = 1
discount = 5 / 100
discount_foo = lambda y: (1-discount)**(y-1)
p = .8

mdp = animal_breeding.farm_problem(
    max_animals,
    animal_price,
    feeding_cost,
    discount,
    p
)

time_horizon = 20
pi, u = mdp.bia(time_horizon)

In [5]:
pd.DataFrame(pi, index = range(1, time_horizon)) #Columns is the current state, row the time points.

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
1,W,1,B,B,B,B,B,B,B,B,B,B,6,7,6,7
2,W,1,B,B,B,B,B,B,B,B,B,B,6,5,6,7
3,W,1,B,B,B,B,B,B,B,B,B,B,6,5,6,7
4,W,1,B,B,B,B,B,B,B,B,B,B,6,7,6,7
5,W,1,B,B,B,B,B,B,B,B,B,B,6,5,6,7
6,W,1,B,B,B,B,B,B,B,B,B,B,6,5,6,5
7,W,1,B,B,B,B,B,B,B,B,B,B,6,7,6,5
8,W,1,B,B,B,B,B,B,B,B,B,B,6,5,6,7
9,W,1,B,B,B,B,B,B,B,B,B,B,6,5,6,5
10,W,1,B,B,B,B,B,B,B,B,B,B,6,7,6,5


In [6]:
pd.DataFrame(u, index = range(1, time_horizon+1))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
1,0.0,10.0,155.465511,167.962389,182.063065,187.842169,197.305367,202.956798,210.488505,214.373285,223.119986,225.665251,230.340469,235.370045,240.542361,245.571937
2,0.0,10.0,147.661654,159.916475,173.723867,179.904369,189.913613,195.115060,202.115505,205.942693,214.669522,217.213372,222.043826,227.624784,233.326328,238.355904
3,0.0,10.0,139.385014,152.230814,165.587890,171.454659,181.616970,187.388676,194.899472,198.496029,206.412356,208.786859,213.618488,219.172726,224.874270,229.903846
4,0.0,10.0,131.386010,143.884765,158.067326,163.820479,173.191631,178.875809,186.447414,190.358501,199.142683,201.691762,206.364054,211.393631,216.404156,221.433732
5,0.0,10.0,123.613702,135.829087,149.648684,155.879912,165.937198,171.070905,177.977300,181.832116,190.620875,193.189403,197.993169,203.651809,209.353352,214.382928
6,0.0,10.0,115.261659,128.201713,141.485930,147.333771,157.566313,163.383346,170.926496,174.523617,182.300440,184.636538,189.471532,195.112977,200.814521,205.893423
7,0.0,10.0,107.272111,119.759046,134.062380,139.796383,149.044676,154.761905,162.387665,166.328026,175.168110,177.723316,182.391081,187.420657,192.242378,197.272660
8,0.0,10.0,99.531812,111.707185,125.522012,131.826236,141.964224,147.023096,153.815521,157.674970,166.547347,169.179400,173.895327,179.684210,185.385754,190.415330
9,0.0,10.0,91.075276,104.145946,117.347495,123.150908,133.468471,139.359826,146.958897,150.558040,158.175539,160.491433,165.266229,170.996710,176.698253,181.922072
10,0.0,10.0,83.131411,95.561242,110.042122,115.776911,124.839373,130.580050,138.271397,142.261435,151.196759,153.762358,158.425015,163.454591,168.027417,173.132892


It should be quite straightforward how the optimal strategy is affected by the environment. Increasing the bulk discount will have an optimal policy rely less on breeding and selling tactically in smaller bulks. 

In [7]:
discount = 20 / 100 #Compared to the 5% discount.

mdp.discount = 20 / 100

pi, u = mdp.bia(time_horizon)

pd.DataFrame(pi, index = range(1, time_horizon))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
1,W,1,B,B,B,1,2,3,2,3,3,3,3,4,3,4
2,W,1,B,B,B,1,2,3,2,3,3,3,3,4,3,4
3,W,1,B,B,B,B,2,3,2,3,3,3,3,4,3,4
4,W,1,B,B,B,1,2,3,2,3,3,3,3,4,3,4
5,W,1,B,B,B,B,2,3,2,3,3,3,3,4,3,4
6,W,1,B,B,B,1,2,3,2,3,3,3,3,4,3,4
7,W,1,B,B,B,B,2,3,2,3,3,3,3,4,3,4
8,W,1,B,B,B,1,2,3,2,3,3,3,3,4,3,4
9,W,1,B,B,B,B,2,3,2,3,3,3,3,4,3,4
10,W,1,B,B,B,1,2,3,2,3,3,3,3,4,3,4


Decreasing the chance of a successfull reproduction will discourage breeding the stock. This is especially seen towards the end of the problem.

In [8]:
mdp.discount = 5 / 100 
mdp.p = .4

pi, u = mdp.bia(time_horizon)

pd.DataFrame(pi, index = range(1, time_horizon))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
1,W,1,B,B,B,B,B,B,B,B,B,5,6,5,6,5
2,W,1,B,B,B,B,B,B,B,B,B,5,6,5,6,5
3,W,1,B,B,B,B,B,B,B,B,B,5,6,5,6,5
4,W,1,B,B,B,B,B,B,B,B,B,5,6,5,6,5
5,W,1,B,B,B,B,B,B,B,B,B,5,6,5,6,5
6,W,1,B,B,B,B,B,B,B,B,B,5,6,5,6,5
7,W,1,B,B,B,B,B,B,B,B,B,5,6,5,6,5
8,W,1,B,B,B,B,B,B,B,B,B,5,6,5,6,5
9,W,1,B,B,B,B,B,B,B,B,B,5,6,5,6,5
10,W,1,B,B,B,B,B,B,B,B,B,5,6,5,6,5


Increasing the marginal feeding cost discourages having a large stock at any time. Breeding only in the event that there is a single pair remaining becomes optimal if the marginal feeding cost is three.

In [9]:
mdp.p = .8
mdp.feeding_cost = 3

pi, u = mdp.bia(time_horizon)

pd.DataFrame(pi, index = range(1, time_horizon))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
1,W,1,B,3,4,4,4,5,6,7,7,8,8,8,9,9
2,W,1,B,3,4,4,4,5,6,7,7,8,8,8,9,9
3,W,1,B,3,4,4,4,5,6,7,7,8,8,8,9,9
4,W,1,B,3,4,4,4,5,6,7,7,8,8,8,9,9
5,W,1,B,3,4,4,4,5,6,7,7,8,8,8,9,9
6,W,1,B,3,4,4,4,5,6,7,7,8,8,8,9,9
7,W,1,B,3,4,4,4,5,6,7,7,8,8,8,9,9
8,W,1,B,3,4,4,4,5,6,7,7,8,8,8,9,9
9,W,1,B,3,4,4,4,5,6,7,7,8,8,8,9,9
10,W,1,B,3,4,4,4,5,6,7,7,8,8,8,9,9


## Infinite horizon problem
Let $\lambda \in [0,1)$ be the discount factor for the infinite horizon problem. Consider as usual
$$
\textstyle 
V^\pi(x) 
=
\mathbb{E}
\left[
\sum_{t = 0}^\infty
\lambda^t r_\pi(X_t) 
\mid
X_0 = x
\right], \, 
x \in E 
$$
In choosing an algorithm to approximate $V^*$, we have choices
- Value iteration algorithm,
- Policy iteration algorithm.

We choose to try out the value iteration algorithm. Recall
- Select a threshold level $\epsilon \in \mathbb{R}_{>0}$ and initial guess $V^0 \in \mathbb{R}^{\mathcal{E}}$,
- In round $n$, calculate $V^{n+1} = \mathcal{L} V^n$.
- If $\left\| V^{n+1} - V^n \right\| < \epsilon (1 - \lambda) / (2\lambda)$, terminate the algorithm.
- $\epsilon$-optimal policy is stationary, deterministic, and choosen as the argument that maximizes the Bellman equation.

We implement the algorithm as so

In [10]:
def via(
        self,
        lmd: float = .9,
        epsilon: float = 0.01,
        v_0: np.ndarray = None
    ) -> list:

    threshold = epsilon * (1 - lmd) / (2 * lmd)
    v_0 = np.zeros(shape = len(self.states)) if v_0 == None else v_0
    n = 0
    v_next = v_0
    stop_condition = True
    epsilon_optimal_policy = np.empty(shape=len(self.states), dtype = object)
    
    #History of the value function approximation.
    data = list()

    while stop_condition:
        
        v_current = v_next 
        v_next = np.zeros(shape = len(self.states), dtype=float)
        n += 1
        for x in self.states:
            tmp = dict()
            for a in self.actions[x]:
                tmp[a] = self.r(x, a)
                for y in self.states:
                    tmp[a] += lmd * self.t(y, x, a) * v_current[y]
            max_key = max(tmp, key=tmp.get)
            max_val = max(tmp.values())
            v_next[x] = max_val
            epsilon_optimal_policy[x] = max_key
        
        data.append(v_next)
        stop_condition = np.linalg.norm(v_current - v_next, ord=1) >= threshold

    history = pd.DataFrame(
        data=data, columns=mdp.states
    )
    history.index += 1

    print("Converged after {n} iterations".format(n=n))
    return [epsilon_optimal_policy, v_next, history]

Considering the same problem as before and default arguments

In [11]:
max_animals = 15 
animal_price = 10
feeding_cost = 1
discount = 5 / 100
discount_foo = lambda y: (1-discount)**(y-1)
p = .8

mdp = animal_breeding.farm_problem(
    max_animals,
    animal_price,
    feeding_cost,
    discount,
    p
)

epsilon_optimal_policy, v_next, history = mdp.via(lmd = .9)

Converged after 104 iterations


In [12]:
epsilon_optimal_policy #Action chosen in state 0, ..., 15

array(['W', 1, 'B', 'B', 'B', 'B', 'B', 'B', 'B', 5, 6, 7, 6, 7, 8, 7],
      dtype=object)

In [13]:
history_melt = history.reset_index() \
               .melt(id_vars="index", value_vars=[*range(0, 16)]) \
               .rename({"index": "Iteration", "value": "Value function", "variable": "State"}, axis=1)

px.line(history_melt, x="Iteration", y="Value function", color="State")

Clearly, given the current problem the convergence rate of the iteration algorithm is logarithmic. Of course we want to stress test the algorithm. We consider increasing the size of the state space $E$.

In [14]:
max_animals = 100 
animal_price = 10
feeding_cost = 1
discount = 5 / 100
discount_foo = lambda y: (1-discount)**(y-1)
p = .8

mdp = animal_breeding.farm_problem(
    max_animals,
    animal_price,
    feeding_cost,
    discount,
    p
)

epsilon_optimal_policy, _, history = mdp.via()

Converged after 123 iterations


In [15]:
epsilon_optimal_policy

array(['W', 1, 'B', 'B', 'B', 'B', 'B', 'B', 'B', 5, 6, 7, 6, 7, 8, 7, 8,
       8, 8, 9, 8, 9, 9, 10, 10, 10, 10, 11, 11, 11, 11, 12, 11, 12, 12,
       13, 13, 13, 14, 13, 14, 14, 14, 14, 15, 15, 15, 15, 15, 16, 16, 16,
       16, 17, 17, 17, 17, 17, 18, 18, 18, 18, 19, 19, 19, 19, 20, 19, 20,
       20, 20, 21, 21, 21, 22, 22, 23, 23, 23, 23, 23, 23, 23, 24, 24, 24,
       25, 25, 25, 26, 26, 26, 27, 27, 27, 27, 28, 28, 28, 29, 29],
      dtype=object)

In [16]:
history_melt = history.reset_index() \
               .melt(id_vars="index", value_vars=[*range(0, max_animals+1)]) \
               .rename({"index": "Iteration", "value": "Value function", "variable": "State"}, axis=1)

px.line(history_melt, x="Iteration", y="Value function", color="State")

Norteworthy, even though we went on to increase the size of the state by more than a factor of three, only 19 extra iterations were needed, and the convergence rate remains logarithmic.

To understand how the discount factor $\lambda \in [0,1)$ affects the $\epsilon$-optimal policy, we make the observation that as $\lambda \to 1^{-}$, the smaller the criteria becomes. Another way of interpreting this is that later rewards carry close to the same weight as early rewards, implying we need more iterations to fully grasp the magnitude of the value function. Therefore, as we will demonstrate, formulating the problem with a small discount factor leads to a faster convergence and a radically different policy.

In [17]:
max_animals = 100 
animal_price = 10
feeding_cost = 1
discount = 5 / 100
discount_foo = lambda y: (1-discount)**(y-1)
p = .8

mdp = animal_breeding.farm_problem(
    max_animals,
    animal_price,
    feeding_cost,
    discount,
    p
)

epsilon_optimal_policy_lambda_small, _, history_lambda_small = mdp.via(lmd = .5)

Converged after 6 iterations


In [18]:
epsilon_optimal_policy_lambda_small

array(['W', 1, 2, 3, 4, 5, 6, 7, 8, 8, 9, 9, 10, 10, 11, 11, 12, 12, 13,
       13, 14, 14, 15, 15, 15, 16, 16, 16, 17, 17, 17, 18, 18, 18, 19, 19,
       19, 20, 20, 20, 21, 21, 21, 21, 22, 22, 22, 23, 23, 23, 23, 24, 24,
       24, 24, 25, 25, 25, 25, 26, 26, 26, 26, 27, 27, 27, 27, 28, 28, 28,
       28, 29, 29, 29, 30, 30, 31, 31, 31, 31, 31, 32, 32, 32, 33, 33, 33,
       34, 34, 35, 35, 35, 35, 36, 37, 38, 38, 39, 63, 67, 71],
      dtype=object)

In [19]:
history_melt = history_lambda_small.reset_index() \
               .melt(id_vars="index", value_vars=[*range(0, max_animals+1)]) \
               .rename({"index": "Iteration", "value": "Value function", "variable": "State"}, axis=1)

px.line(history_melt, x="Iteration", y="Value function", color="State")

Note as expected that the convergence is fast, as it should be by the threshold being more liberal. Observing the $\epsilon$-optimal policy, little care is taken into breeding the animal population, implying that, as expected, later rewards from selling off the population matter little.

## Q-Learning

Recall the $Q$-function $Q(x, a) = V^{\pi^a} (x)$ where $\pi_0^a(x) = a$ and $\pi_t^a(x) = \pi_t(x)$ for $x \in E$ and $a \in A(x)$. Under our standing assumptions, $V^*(x) = \max_{a \in A(x)} Q^* (x, a)$ for all $x \in E$. $Q$-learning as a dynamical system is formulated as 
$$
Q_{n+1}(x,a) 
=
(1-\alpha_n) Q_n(x,a)
+
\alpha_n \left[
    r(x,a) + \lambda \max_{b \in A(x)}(Y_{n+1}(x, a), b)
\right]
$$
where 
- $(Y_n)_{n \in \mathbb{N}}$ defines a sequence s.t. $Y_n(x,a) \sim p (\, \cdot \, \mid x, a)$ for any state-action pair $(x,a) \in E \times A$.
- $(\alpha_n)_{n \in \mathbb{N}}$ is a sequence s.t. $\alpha_n \to 0$.
    - This sequence should never converge too fast to zero. This we express as the criteria $\sum_{n \in \mathbb{N}} \alpha_n = \infty$ and $\sum_{n \in \mathbb{N}} \alpha_n^2 < \infty$. One example would be the harmonic series. The condition expressess that as iterations progress, the more we rely on the previous iterations compared to optimizing the Bellman equation.
    
The convergence of $Q_n \to Q^*$ a.s. follows from the fact that the discrete problem can be embedded into a continous general dynamic system. This we may formulate into an algorithm:
- Select some $n^* \in \mathbb{N}$ and $Q_0 \in \mathbb{R}^{\mathcal{E} \times \mathcal{A}}$.
- For $n = 1, \ldots, n^*$, evaluate $Q_{n+1}(x,a)$ for any state-action pair $(x,a) \in E \times A$ using the update scheme presented before.

Alternatively, one could use a criteria similar to that of the iterated value function since we work in both cases with contractions by the Bellman equation. (We do not implement that here.)

Finding a close to optimal policy by $Q$-learning can be done by selecting $\pi^*(x) = \text{argmax}_{a \in A(x)} Q_{n^*}(x, a)$ for all $x \in E$. 

$Q$-learning is generally a model-free technique. Therefore, having observed enough simulations of the Markov chain evaluated in different pure, deterministic policies yields an estimate of the distribution of the transition probabilities. We do not estimate them here, but we will simulate according to the actual transition probabilities.

$Q$-learning can be implemented as follows

In [20]:
# Function to simulate Y_n(x, a) ~ p( \mid x, a)
def y(self, x, a):
    probability = [0 for _ in range(len(self.states))]
    for state in self.states:
        probability[state] = self.t(state, x, a)
    return np.random.choice(self.states, p=probability)

# Function to evaluate the Q-learning scheme.
def q(
    self,
    lmd: float = .5,
    threshold_n: int = 100,
    alpha = lambda k: 1/k
) -> list():

    n = 1

    value_function = np.empty(shape=len(self.states), dtype=float)
    policy = np.empty(shape=len(self.states), dtype=object)
    
    data = np.empty(shape=(threshold_n, len(self.states)), dtype=float)
    history = pd.DataFrame(
        data=data, columns=self.states, index=range(1,threshold_n+1)
    )
    del data

    q_current = dict()
    for state in self.states:
        q_current[state] = np.array([0 for _ in self.actions[state]])

    while n < threshold_n:
        
        q_next = dict()
        for state in self.states:
            tmp1 = (1-alpha(n)) * q_current[state]
            tmp2 = np.empty(shape=tmp1.shape, dtype = float)
            i = 0 
            for action in self.actions[state]:
                tmp2[i] = self.r(state, action) + lmd * np.max(q_current[self.y(state, action)]) 
                i += 1
            tmp2 *= alpha(n)
            tmp = tmp1 + tmp2
            q_next[state] = tmp

        q_current = q_next
        for state in self.states:
            policy[state] = self.actions[state][np.argmax(q_current[state])]
            value_function[state] = max(q_current[state])
        history.loc[n,] = value_function
        n += 1

    return [policy, value_function, history]

Implementing for the same problem as previously

In [21]:
max_animals = 4 
animal_price = 10
feeding_cost = 1
discount = 5 / 100
discount_foo = lambda y: (1-discount)**(y-1)
p = .8

mdp = animal_breeding.farm_problem(
    max_animals,
    animal_price,
    feeding_cost,
    discount,
    p
)


epsilon_optimal_policy, v_next, history = mdp.via(lmd=.5)
q_optimal_policy, q_v, q_history = mdp.q(threshold_n=100, lmd=.5)

Converged after 2 iterations


In [22]:
print(
    *["Optimal policy by Q-learning", q_optimal_policy,
    "Optimal policy by value iteration algorithm", epsilon_optimal_policy],
    sep="\n"
)

Optimal policy by Q-learning
['W' 1 2 3 4]
Optimal policy by value iteration algorithm
['W' 1 2 3 4]


Note that the policies are identitcal.

In [26]:
q_optimal_policy, q_v, q_history = mdp.q(threshold_n=500, lmd=.9)
print(
    *["Value function by Q-learning", q_v,
    "Value function by value iteration algorithm", v_next],
    sep="\n"
) 

Value function by Q-learning
[ 0.         10.         20.99268067 27.23788553 35.48090633]
Value function by value iteration algorithm
[ 0.    10.    19.    27.075 34.295]
